In [ ]:
!pip install pydub
!pip install resemblyzer
!pip install spectralcluster
!pip install pytube

     |████████████████████████████████| 15.7 MB 144 kB/s 
     |████████████████████████████████| 66 kB 4.8 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72399 sha256=87386ea08abef8b1c8ff3c0c768da23b21c88a849da187200bb8bb3c64d972f9
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=1d8c1679d83af91dee3bcb791dd70e3310987f17851d389550fb11bda1a1797e
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built webrtcvad typing


     |████████████████████████████████| 56 kB 3.4 MB/s 


In [ ]:
import os
from pytube import YouTube

yt = YouTube('https://www.youtube.com/watch?v=rXqh6AcIb-c')
yt.streams.first().download()
os.rename(yt.streams.first().default_filename, 'grande_fallon_1.mp4')

In [ ]:
!ffmpeg -i grande_fallon_1.mp4 -ab 160k -ac 2 -ar 44100 -vn grande_fallon_1.wav

In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = 'grande_fallon_1.wav'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.03 seconds.
(5802, 256)


In [ ]:
from spectralcluster import SpectralClusterer
from spectralcluster import RefinementOptions

ref_opts = RefinementOptions(
    p_percentile=0.90,
    gaussian_blur_sigma=1)


clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    refinement_options = ref_opts)

labels = clusterer.predict(cont_embeds)

In [ ]:
def create_labelling(labels,wav_splits):
    from resemblyzer.audio import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [ ]:
import csv 
with open('labels_Fallon_2.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    
    write.writerows(labelling)